In [1]:
#Import packages
import pandas as pd
from pandas import DataFrame
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
import requests

In [4]:
#Define where's the CSV file
filePath = 'C:/Users/LuisRicardoFerraz/Documents/personal/projects/book-similarity/test/tutorial/tutorial/spiders/dataset-clean/'

In [10]:
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'datasetSemDuplicatas.csv')
df = df.drop(['Unnamed: 0'],1)

In [2]:
#Count number of characters in each string
def countCharactersInEachString():
    charactersCount = []
    for synopsis in df['sinopse']:
        charactersCount.append(len(str(synopsis)))
    return charactersCount

In [3]:
#Count number of words in each string
def countWordsInEachString():
    wordsCount = []
    for synopsis in df['sinopse']:
        words = word_tokenize(str(synopsis))
        wordsCount.append(len(words))
    return wordsCount

In [5]:
#Extract some basic statistics about synopsis
def extractBasicStatisticsAboutSynopsis(arrayOfSizes):
    synopsisLength = np.array(arrayOfSizes)
    basicStatistics = "\t"   
    basicStatistics += "Arithmetic Mean: " + str(np.mean(synopsisLength)) + "\n\t"
    basicStatistics += "Median: " + str(np.median(synopsisLength)) + "\n\t"
    basicStatistics += "Highest Value: " + str(np.max(synopsisLength)) + "\n\t"
    basicStatistics += "Lowest Value: " + str(np.min(synopsisLength)) + "\n\t"
    basicStatistics += "Mid-range: " + str((np.max(synopsisLength)-np.min(synopsisLength))/2) + "\n\t"
    basicStatistics += "Variance: " + str(np.var(synopsisLength,ddof=1)) + "\n\t"
    basicStatistics += "Standard Deviation: " + str(np.std(synopsisLength,ddof=1)) + "\n\t"
    return basicStatistics

In [4]:
def extractPercentilesAboutSynopsis(arrayOfSizes):
    synopsisLength = np.array(arrayOfSizes)
    percentiles = "Percentiles: " + "\n\t"
    count = 0.5
    while count <= 100:
        percentiles += "Percentile " + str(count) + ": " + str(np.percentile(arrayOfSizes,count)) + "\n\t"
        count += 0.5
    return percentiles

In [6]:
#Extract some statistics about this Dataset
def extractStatisticsFromSynopsis(title):
    log = title + "\n\n"
    log += "Shape of Dataset: " + str(df.shape[0]) + " rows and " + str(df.shape[1]) + " columns" + "\n\n"
    log += "Statistics of Synopsis Length (Characters):" + "\n"
    charactersCount = countCharactersInEachString()
    log += extractBasicStatisticsAboutSynopsis(charactersCount) + "\n"
    log += extractPercentilesAboutSynopsis(charactersCount) + "\n"
    log += "Statistics of Synopsis Length (Words):" + "\n"
    wordsCount = countWordsInEachString()
    log += extractBasicStatisticsAboutSynopsis(wordsCount) + "\n"
    log += extractPercentilesAboutSynopsis(wordsCount) + "\n"
    return log

In [7]:
df['sinopse'] = [sinopse.lower() for sinopse in df['sinopse']]
df['sinopse']

0        uma jovem garota brasileira resolve se aventur...
1        "resgate" foi a primeira palavra que ouvi quan...
2        cage: substantivo: cela, prisão.  verbo: prend...
3        feyre has returned to the spring court, determ...
4        quebrado. irritado. abandonado. era assim que ...
5        uma distopia atual, próxima dos dias de hoje, ...
6        as histórias de contos de fadas que as criança...
7        maitê passos é uma garota linda, de dezessete ...
8        existe uma forma de leveza e de graça no simpl...
9        um thriller sobre a luta secreta entre o bem e...
10       as crônicas de gelo e fogo. jon snow, daenerys...
11       duke é um homem simples com uma vida modesta, ...
12       o oitavo volume de wild cards injeta ainda mai...
13       diana prince, mais conhecida como mulher-marav...
14       na são paulo do início do século xx, a jovem g...
15       antes de se tornar a mulher-maravilha, ela era...
16       podemos até não viver no paraíso, mas passamos.

In [1]:
filePath = 'C:/Users/LuisRicardoFerraz/Documents/personal/projects/book-similarity/test/tutorial/tutorial/spiders/synopsis-cleaning/'

In [19]:
with open(filePath+'logDatasetComSinopsesEmMinusculas.txt','w') as log:
    log.write(extractStatisticsFromSynopsis("Statistics of Dataset - Lower case on synopsis"))
df.to_csv(filePath+'dataSetComSinopsesEmMinusculas.csv')

In [8]:
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'dataSetComSinopsesEmMinusculas.csv')
df = df.drop(['Unnamed: 0'],1)

In [9]:
def findInSynopsis(array_tmp, pattern):
    for text in array_tmp:
        occurrences = re.findall(pattern,text)
        if len(occurrences) > 0:
            print(occurrences)

In [10]:
# remover URLs, hashtags e tags de marcação
# colocar um espaço entre pontos e palavras
# remover números
# remover pontuação e caracteres especiais (travessão, reticências, aspas francesas)
# tokenizar

def removeUrlsFromSynopsis(text):
    text = re.sub('http\S+','',text)
    text = re.sub('www.\S+','',text)
    return text

def removeHashtagsFromSynopsis(text):
    text = re.sub('#\w+','',text)
    return text

def removeProfileTagsFromSynopsis(text):
    text = re.sub('@\w+','',text)
    return text

def insertSpaceBetweenDotsAndWords(text):
    occurrences = re.findall('\.\w+',text)
    for string in occurrences:
        fixing = string.replace('.','. ')
        text = text.replace(string,fixing)
    return text

def removeNumbersFromSynopsis(text):
    text = re.sub('[0-9]+','',text)
    return text

def removeSpecialCharacters(text):
    text = re.sub('[－⸻\^\?\.֊`º︱﹣®‘%\\―〝´~゠\־⹀〞\$〰\)»;/ª}&︲•<:«°"|!’§⸚｢\+\]‑⸺=《\*(–,_—”…\-᐀⸗〜>\[‐᠆@‒“﹘·\'{]+',' ', text)
    return text

def removeDieresisFromSynopsis(text):
    text = re.sub('[Ä]','A',text)
    text = re.sub('[Ë]','E',text)
    text = re.sub('[Ï]','I',text)
    text = re.sub('[Ö]','O',text)
    text = re.sub('[Ü]','U',text)
    text = re.sub('[ä]','a',text)
    text = re.sub('[ë]','e',text)
    text = re.sub('[ï]','i',text)
    text = re.sub('[ö]','o',text)
    text = re.sub('[ü]','u',text)
    return text

def removeExtraSpacesFromSynopsis(text):
    text = re.sub('[ ]+',' ',text)
    return text

In [14]:
df.shape

(10299, 27)

In [13]:
df = df[df['sinopse'].map(len) > 140]

In [15]:
with open(filePath+'logDatasetComSinopsesDeTamanhoFiltrado.txt','w') as log:
    log.write(extractStatisticsFromSynopsis("Statistics of Dataset - Minimum Length For Synopsis"))
df.to_csv(filePath+'dataSetComSinopsesDeTamanhoFiltrado.csv')

In [16]:
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'dataSetComSinopsesDeTamanhoFiltrado.csv')
df = df.drop(['Unnamed: 0'],1)

In [17]:
df['sinopse'] = [removeUrlsFromSynopsis(text) for text in df['sinopse']]

In [18]:
with open(filePath+'logDatasetComSinopsesSemUrls.txt','w') as log:
    log.write(extractStatisticsFromSynopsis("Statistics of Dataset - Synopsis without URLs"))
df.to_csv(filePath+'dataSetComSinopsesSemUrls.csv')

In [19]:
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'dataSetComSinopsesSemUrls.csv')
df = df.drop(['Unnamed: 0'],1)

In [20]:
df['sinopse'] = [removeHashtagsFromSynopsis(text) for text in df['sinopse']]
with open(filePath+'logDatasetComSinopsesSemHashtags.txt','w') as log:
    log.write(extractStatisticsFromSynopsis("Statistics of Dataset - Synopsis without Hashtags"))
df.to_csv(filePath+'dataSetComSinopsesSemHashtags.csv')
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'dataSetComSinopsesSemHashtags.csv')
df = df.drop(['Unnamed: 0'],1)

In [21]:
df['sinopse'] = [removeProfileTagsFromSynopsis(text) for text in df['sinopse']]

In [22]:
with open(filePath+'logDatasetComSinopsesSemProfileTags.txt','w') as log:
    log.write(extractStatisticsFromSynopsis("Statistics of Dataset - Synopsis without Profile Tags"))
df.to_csv(filePath+'dataSetComSinopsesSemProfileTags.csv')
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'dataSetComSinopsesSemProfileTags.csv')
df = df.drop(['Unnamed: 0'],1)

In [23]:
df['sinopse'] = [insertSpaceBetweenDotsAndWords(text) for text in df['sinopse']]
df['sinopse'] = [removeNumbersFromSynopsis(text) for text in df['sinopse']]
with open(filePath+'logDatasetComSinopsesSemNumeros.txt','w') as log:
    log.write(extractStatisticsFromSynopsis("Statistics of Dataset - Synopsis without Numbers"))
df.to_csv(filePath+'dataSetComSinopsesSemNumeros.csv')
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'dataSetComSinopsesSemNumeros.csv')
df = df.drop(['Unnamed: 0'],1)

In [24]:
df['sinopse'] = [removeSpecialCharacters(text) for text in df['sinopse']]
df['sinopse'] = [removeDieresisFromSynopsis(text) for text in df['sinopse']]
with open(filePath+'logDatasetComSinopsesSemCaracteresEspeciais.txt','w') as log:
    log.write(extractStatisticsFromSynopsis("Statistics of Dataset - Synopsis without Special Characters"))
df.to_csv(filePath+'dataSetComSinopsesSemCaracteresEspeciais.csv')
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'dataSetComSinopsesSemCaracteresEspeciais.csv')
df = df.drop(['Unnamed: 0'],1)

In [25]:
df['sinopse'] = [removeExtraSpacesFromSynopsis(text) for text in df['sinopse']]
with open(filePath+'logDatasetComSinopsesSemExcessoDeEspacos.txt','w') as log:
    log.write(extractStatisticsFromSynopsis("Statistics of Dataset - Synopsis Trimmed"))
df.to_csv(filePath+'dataSetComSinopsesSemExcessoDeEspacos.csv')
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'dataSetComSinopsesSemExcessoDeEspacos.csv')
df = df.drop(['Unnamed: 0'],1)

In [2]:
filePath = 'C:/Users/LuisRicardoFerraz/Documents/personal/projects/book-similarity/test/tutorial/tutorial/spiders/synopsis-preprocessing/'

In [15]:
with open(filePath+'logDatasetComSinopsesLimpas.txt','w') as log:
    log.write(extractStatisticsFromSynopsis("Statistics of Dataset - Synopsis Cleaned"))
df.to_csv(filePath+'dataSetComSinopsesLimpas.csv')

In [15]:
#Import CSV file to a DataFrame
df = pd.read_csv(filePath+'dataSetComSinopsesLimpas.csv')
df = df.drop(['Unnamed: 0'],1)

In [16]:
#There are problems with «» and ÄËÏÖÜäëïöü and the size of the synopsis and ´ ª ° extra spaces ‘ ?? § and UTF-8 (must convert to string first) and · ®
#Limit for PortService-Br analysis: 7000 characters in each synopsis
def identifyUnknownWords(text):
    log = ''
    try:
        payload = {'texto': str(text)}
        response = requests.get('http://portservice.pythonanywhere.com/analise/morfossintatica/freeling', params=payload).json()
        string = []
        for word in response:
            string.append('' if word['lemma'] != '<unknown>' else word['token'])
        log += str(" ".join(set(string))) + "\n"
    except:
        print(text)
    return log

In [17]:
#This is a log to identify words which PortService-Br doesn't recognize
#If the word is from Portuguese and is mispelled, or it is not from Portuguese language, it will be shown in the .txt file.
#If there's anything wrong with the API request, the synopsis will be shown here.
#with open(filePath+'logDatasetComPalavrasDesconhecidasIdentificadas.txt','w', encoding='utf-8') as log:
#    for text in df['sinopse']:
#        log.write(identifyUnknownWords(text))

In [53]:
#This code helps on identifying words which have the same letter repeated 3+ times
#They are part of not-in-vocabulary words
#There are only few words that match this case, so the best I can do is to ignore them and remove them along with the stopwords
#array_tmp = df['sinopse']
#for letter in ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']:
#    findInSynopsis(array_tmp, '\S+['+letter+']{3,}\S+')